# Data Analys 2 (RNN)
    This notebook is based on the walkDatas_Analysis and proposes another approach for the neural network: the recursive neural network (RNN). More precisely, it uses the LSTM (Long Short-Term Memory) NN.
    

In [5]:
# import math
import pandas as pd
# import numpy as np
# import re
# import csv
# from datetime import datetime
# import folium
# from folium import plugins
import matplotlib.pyplot as plt
# import os

# from keras.models import Sequential
# from keras.layers import Dense
# from keras.optimizers import Adam

# from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score

import numpy as np
from tensorflow.keras import Input
from tensorflow.keras.layers import LSTM,Dense,Bidirectional
from tensorflow.keras.models import Model

#pip install -U numpy==1.18.5

file_path = "C:\\Users\\jerem\\Documents\\TrundleWheel\\Data\\User1_Jeremy_Bezancon\\" #file path of the txt file

## <font color='red'> Multivariate LSTM model </font>


In [70]:
file_path = "C:\\Users\\jerem\\Documents\\TrundleWheel\\Data\\User1_Jeremy_Bezancon\\" #file path of the txt file
data = pd.read_csv(file_path+'all_datas_2'+'.csv', sep=';')

row_s, row_e = 0, int(len(data)/10)      # first row, last row
column_s, column_e = 1, 11   # first column, last column (last one isn't taken into account)
# Columns 1 to 9 correspond to the accelereation datas
# Column 10 correspond to steps data
# Column 11 correspond to distance data
print("train data length = ",row_e)

dataset = data.iloc[row_s:row_e, list(range(column_s, column_e)) + [11]].values
# print(dataset)

# reshape input values between 0 and 1:
# columns_to_scale = dataset[:, column_s:column_e]
# scaler = MinMaxScaler(feature_range=(0, 1))
# scaled_columns = scaler.fit_transform(columns_to_scale)
# dataset[:, column_s:column_e] = scaled_columns

print(dataset)

train data length =  358074
[[  0.3         -1.4         -0.7        ... -19.           1.
    1.00625   ]
 [  0.3         -1.1         -1.2        ... -15.           1.
    1.01875   ]
 [  0.3         -1.          -1.3        ... -11.3          1.
    1.03125   ]
 ...
 [ -3.8        -10.3         -1.5        ...  63.6        109.46218487
  111.94029851]
 [ -3.         -10.1         -0.9        ...  63.6        109.4789916
  111.97014925]
 [ -1.5        -10.           0.9        ...  87.7        109.50420168
  112.01428571]]


In [71]:
def split_sequence_total(sequences,n_steps):
    X,y = list(),list()
    
    for i in range(len(sequences)):
        end_ix = i +n_steps
        if end_ix >len(sequences):
            break
            
        seq_x,seq_y = sequences[i:end_ix,:-1],sequences[end_ix-1,-1]
        X.append(seq_x)
        y.append(seq_y) 
        
    return np.array(X),np.array(y)

def split_sequence_difference(sequences,n_steps):
    X,y = list(),list()
    
    old_sequences_y = 0
    old_sequences_steps = np.zeros(n_steps)
    
    for i in range(len(sequences)):
        end_ix = i +n_steps
        if end_ix >len(sequences):
            break
            
        seq_x,seq_y = sequences[i:end_ix,:-1].copy(),sequences[end_ix-1,-1]-old_sequences_y       
        
        seq_x[0:n_steps,-1] = seq_x[0:n_steps,-1] - old_sequences_steps
    
        old_sequences_y = sequences[i,-1]
        old_sequences_steps = np.full(n_steps, sequences[i,-2])

        X.append(seq_x)
        y.append(seq_y)
        
    return np.array(X),np.array(y)

In [72]:
n_steps = 5

X,y = split_sequence_difference(dataset,n_steps)
print(X.shape,y.shape)

# for i in range(100):
#     print(X[i+1000],y[i+1000])

(358070, 5, 10) (358070,)


#### <font color='chocolate'>define model</font>

In [73]:
n_features = X.shape[2]
x0 = Input(shape=(n_steps,n_features))
x1 = LSTM(50,activation='relu')(x0)
output = Dense(1)(x1)

model = Model(inputs = x0,outputs = output)
model.compile(optimizer='adam',loss='mse')

In [ ]:
hist = model.fit(X,y,epochs=200,verbose=1)

Epoch 1/200
11190/11190 [==============================] - 37s 3ms/step - loss: 39.1560
Epoch 2/200
11190/11190 [==============================] - 36s 3ms/step - loss: 38.8532
Epoch 3/200
11190/11190 [==============================] - 37s 3ms/step - loss: 26.6915
Epoch 4/200
11190/11190 [==============================] - 38s 3ms/step - loss: 26.9041
Epoch 5/200
11190/11190 [==============================] - 41s 4ms/step - loss: 24.3506
Epoch 6/200
11190/11190 [==============================] - 40s 4ms/step - loss: 13.8049
Epoch 7/200
11190/11190 [==============================] - 39s 3ms/step - loss: 5.0864
Epoch 8/200
11190/11190 [==============================] - 40s 4ms/step - loss: 5.8055
Epoch 9/200
11190/11190 [==============================] - 39s 4ms/step - loss: 6.9650
Epoch 10/200
11190/11190 [==============================] - 39s 3ms/step - loss: 9.4462
Epoch 11/200
11190/11190 [==============================] - 39s 4ms/step - loss: 18.5050
Epoch 12/200
11190/11190 [========

In [ ]:
data_valid = pd.read_csv(file_path+'normal_speed_valid_conditioned_2'+'.csv', sep=';') # Normal speed
# data_valid = pd.read_csv(file_path+'mix_of_everything_valid_conditioned_2'+'.csv', sep=';') # mix of every speed

y_pred_tab = []
time = []
valid_distance = []

# Use this code if you want to make a prediction using the total distance and steps:
#__________________________________________________________________________________
# row_e = len(data_valid)
# N = 100 # number of valid data
# step = int(row_e/N)
# print("length of valid data:",row_e)

# for row in range(0,row_e,step):

    
#     selected_columns = data_valid.iloc[row:row + n_steps, column_s:column_e]    
    
#     x_input = selected_columns
#     x_input = x_input.reshape((1,step,n_features))

#     y_pred = model.predict(x_input,verbose=0) # for total distance and steps
    
#     y_pred_tab.append(y_pred[0,0])
#     time.append(data.iloc[row,0])
#     valid_distance.append(data_valid.iloc[row,11])
#__________________________________________________________________________________

# Use this code if you want to make a prediction using steps difference and distance difference between two samples
#__________________________________________________________________________________
old_y_pred = 0 # last prediction
old_sequences_steps = np.zeros(n_steps)
step = n_steps # it's necessary in this case
row_e = len(data_valid) - (len(data_valid)%step)
row_e = 10000
print("length of valid data:",row_e)

for row in range(0,row_e,step):

    selected_columns = data_valid.iloc[row:row + step, column_s:column_e].values.copy()
    selected_columns[:,-1] = selected_columns[:,-1] - old_sequences_steps
    old_sequences_steps = np.full(step, data_valid.iloc[row+step-1, -2])
    
    x_input = selected_columns
    x_input = x_input.reshape((1,step,n_features))
    
    distance_interval=model.predict(x_input,verbose=0)
    y_pred = distance_interval + old_y_pred
    old_y_pred = y_pred
    
    y_pred_tab.append(y_pred[0,0])
    time.append(data.iloc[row,0])
    valid_distance.append(data_valid.iloc[row,11])
#__________________________________________________________________________________


plt.plot(time, valid_distance, 'r-', label='Validation data', linewidth=0.5)
plt.plot(time, y_pred_tab, 'b-', label='Prediction',linewidth=0.5)
plt.title('Prediction and Validation Data')
plt.xlabel('time (ms)')
plt.ylabel('Distance (m)')
plt.legend()
plt.show()

In [ ]:
# Print the performance of the neural network
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Calculate statistics
mse = mean_squared_error(valid_distance, y_pred_tab)
rmse = np.sqrt(mse)
mae = mean_absolute_error(valid_distance, y_pred_tab)
r2 = r2_score(valid_distance, y_pred_tab)
# mpe = np.mean((valid_distance - y_pred_tab) / valid_distance) * 100

print(f'RMSE: {rmse:.4f}') # average error
print(f'MAE: {mae:.4f}') # mean absolute error
print(f'R²: {r2:.4f}') # determination coefficient (1 is pefect match)
# print(f'MPE: {mpe:.4f}%') # mean percentage error

totat_distance_error = abs(valid_distance[-1]-y_pred_tab[-1])
print("totat distance error:",totat_distance_error)
print("distance max:",valid_distance[-1])

In [ ]:
train_loss = hist.history['loss']
ep = range(1,len(train_loss)+1)

plt.plot(ep,train_loss,'b-',label='train_loss')
plt.title('Training and validation loss')
plt.legend()
plt.xlabel('ep')
plt.ylabel('loss')